# Analyzing Time Series Data

-----

In this notebook, we introduce how to work efficiently with time series data in Python, by using the Pandas library.

-----


## Table of Contents


[](#)


[](#)


[](#)


-----

Before proceeding with the rest of this notebook, we first include the notebook setup code.

-----

In [1]:
# Set up Notebook

% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# These two lines suppress warnings that sometimes
# occur when making visualizations
import warnings
warnings.filterwarnings('ignore')

# Set global figure properties
import matplotlib as mpl
mpl.rcParams.update({'axes.titlesize' : 20,
                     'axes.labelsize' : 18,
                     'legend.fontsize': 16})

# Set default Seaborn plotting style
sns.set_style('white')

-----

[[Back to TOC]](#Table-of-Contents)

## XXX

TEXT

-----



-----

<font color='red' size = '5'> Student Exercise </font>


In the preceding cells, we introduced XXX and used this approach to fit a model to data with uncertainty. Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Change the 
1. Change the 
2. Try using a
3. Try changing 

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

2. Introduction to [working with time series data][pat] by using Pandas
2. Worked example of using [Pandas for time series analysis][pate]
2. Analyzing stock data with [Pandas][nbta]
3. [Financial analysis][fapt] by using Pandas
3. [Financial analysis with Google trends][fapt2] by using Pandas
4. [Time series plots][tspp] by using Python
5. Somewhat dated discussion of using Pandas for [time series analysis][pbts]
5. [Using Pandas for time series analysis][pt]
6. Time series analysis in [Python][tspr] and R

-----

[pat]: http://www.vincentgregoire.com/introduction-to-pandas/

[pate]: http://www.vincentgregoire.com/crsp-example-dividends/

[nbta]: http://nbviewer.jupyter.org/github/jdwittenauer/ipython-notebooks/blob/master/notebooks/misc/TimeSeriesStockAnalysis.ipynb

[fapt]: https://github.com/twiecki/financial-analysis-python-tutorial/blob/master/1.%20Pandas%20Basics.ipynb

[fapt2]: https://github.com/twiecki/financial-analysis-python-tutorial/blob/master/2.%20Pandas%20replication%20of%20Google%20Trends%20paper.ipynb

[tspp]: http://www.blackarbs.com/blog/advanced-time-series-plots-in-python/1/6/2017

[pbts]: http://earthpy.org/pandas-basics.html

[pt]: http://www.marcelscharth.com/python/time.html

[tspr]: http://connor-johnson.com/2014/11/23/time-series-forecasting-in-python-and-r/

[tsd]: https://www.bundesbank.de/Navigation/EN/Statistics/Time_series_databases/Real_Time_Data/realtime_list_node.html?listId=rt_s311_m.de.n.i.io1.acm03.c.i

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode